# Tarea M5 MPAD de Zandalinas_Victor

#### Importar pandas y las librerías utilizadas para resolver la tarea.

In [1]:
# Importe todas las librerias que se usan en esta notebook
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML
import time
import matplotlib.pyplot as plt
from mplsoccer import Pitch, FontManager
import seaborn as sns
import numpy as np

# 1. Cafecito API by Sports Data Campus
 1.0.1 

 
[ Base URL: https://api-cafecito.onrender.com/ ]


In [2]:
# Cuente y describa que api eligió
# Configuración base para la API
BASE_URL = "https://api-cafecito.onrender.com"
AUTH_TOKEN = "EAAHlp1ycWFIBOzFZASIPjVtB1n30C8jUBKHo"

# Cabeceras para las peticiones
headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'Content-Type': 'application/json'
}


# 2. Realice las siguientes tareas, creando funciones para cada paso principal:

### a) Análisis y Planificación (10 puntos)

* Analice la estructura y documentación de la API o conjunto de datos elegido.
* Identifique los endpoints relevantes o los datos específicos a extraer.
* Planifique la estrategia de adquisición de datos, considerando límites de tasa si es aplicable.
* Documente las consideraciones éticas y legales del uso de la API o datos.

In [3]:
##1.Función auxiliar para hacer peticiones a la API
def make_api_request(endpoint):
    """
    Realiza una petición GET a la API y devuelve los datos JSON.
    
    Args:
        endpoint (str): El endpoint de la API a consultar
        
    Returns:
        dict: Los datos recibidos de la API en formato JSON
    """
    url = f"{BASE_URL}{endpoint}"
    response = requests.get(url, headers=headers)
    
    # Verificar si la petición fue exitosa
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al realizar la petición: {response.status_code}")
        print(response.text)
        return None

In [4]:
# Mostrar todos los endpoints disponibles en la API MPAD Cafecito y su propósito
api_structure = [
    {"endpoint": "/competitions", "descripción": "Devuelve todas las competiciones disponibles", "uso": "Obtener lista de competiciones"},
    {"endpoint": "/features/qualifiers", "descripción": "Devuelve la lista de qualifiers", "uso": "Obtener metadatos sobre qualifiers"},
    {"endpoint": "/features/typeId", "descripción": "Devuelve la lista de registros extraída de typeId", "uso": "Obtener metadatos sobre tipos de eventos"},
    {"endpoint": "/match/base/{match_id}", "descripción": "Devuelve la información base del partido", "uso": "Obtener datos básicos de un partido específico"},
    {"endpoint": "/match/events/{match_id}", "descripción": "Obtiene todos los eventos del partido", "uso": "Obtener eventos detallados de un partido específico"},
    {"endpoint": "/match/formationIdNameMappings/{match_id}", "descripción": "Devuelve la lista de formationIdNameMappings para el partido", "uso": "Obtener mapeos de formaciones"},
    {"endpoint": "/match/formations/{match_id}", "descripción": "Devuelve todas las formaciones del equipo local y del equipo visitante", "uso": "Obtener formaciones tácticas"},
    {"endpoint": "/match/incidentEvents/{match_id}", "descripción": "Devuelve los incidentEvents del partido", "uso": "Obtener eventos incidentales"},
    {"endpoint": "/match/matchCentreEventTypeJson/{match_id}", "descripción": "Devuelve la lista de matchCentreEventTypeJson para el partido", "uso": "Obtener tipos de eventos del centro de partido"},
    {"endpoint": "/match/players/{match_id}", "descripción": "Devuelve la lista de jugadores que participaron en el partido", "uso": "Obtener información de jugadores"},
    {"endpoint": "/match/stats/{match_id}", "descripción": "Devuelve los stats del equipo local y visitante", "uso": "Obtener estadísticas del partido"},
    {"endpoint": "/match/{match_id}", "descripción": "Devuelve el contenido completo del partido en crudo", "uso": "Obtener todos los datos del partido"},
    {"endpoint": "/matches", "descripción": "Trae todos los partidos disponibles", "uso": "Obtener lista de todos los partidos"},
    {"endpoint": "/matches/competition/{competition}", "descripción": "Devuelve los partidos filtrados por competición", "uso": "Obtener partidos de una competición específica"},
    {"endpoint": "/matches/competition/{competition}/season/{season}", "descripción": "Trae los partidos de una competición y una temporada específicas", "uso": "Filtrar partidos por competición y temporada"},
    {"endpoint": "/teams", "descripción": "Devuelve la lista de equipos", "uso": "Obtener información de equipos"}
]

# Mostrar la estructura de la API en un DataFrame
api_structure_df = pd.DataFrame(api_structure)
print("Estructura de la API MPAD Cafecito:")
display(api_structure_df)

Estructura de la API MPAD Cafecito:


,endpoint,descripción,uso
0,/competitions,Devuelve todas las competiciones disponibles,Obtener lista de competiciones
1,/features/qualifiers,Devuelve la lista de qualifiers,Obtener metadatos sobre qualifiers
2,/features/typeId,Devuelve la lista de registros extraída de typeId,Obtener metadatos sobre tipos de eventos
3,/match/base/{match_id},Devuelve la información base del partido,Obtener datos básicos de un partido específico
4,/match/events/{match_id},Obtiene todos los eventos del partido,Obtener eventos detallados de un partido espec...
5,/match/formationIdNameMappings/{match_id},Devuelve la lista de formationIdNameMappings p...,Obtener mapeos de formaciones
6,/match/formations/{match_id},Devuelve todas las formaciones del equipo loca...,Obtener formaciones tácticas
7,/match/incidentEvents/{match_id},Devuelve los incidentEvents del partido,Obtener eventos incidentales
8,/match/matchCentreEventTypeJson/{match_id},Devuelve la lista de matchCentreEventTypeJson ...,Obtener tipos de eventos del centro de partido
9,/match/players/{match_id},Devuelve la lista de jugadores que participaro...,Obtener información de jugadores


In [5]:
# Explorar los metadatos disponibles de la API para entender mejor su estructura
print("\nExplorando los metadatos de la API para entender su estructura:")

# Explorar qualifiers
qualifiers = make_api_request("/features/qualifiers")
if qualifiers:
    print("\n1. Qualifiers:")
    if isinstance(qualifiers, list):
        print(f"   - {len(qualifiers)} qualifiers disponibles")
        if len(qualifiers) > 0:
            print("   - Ejemplo de qualifier:")
            print(f"     {json.dumps(qualifiers[0], indent=2)}")
    else:
        print("   - Estructura no es una lista como se esperaba")

# Explorar typeId
type_ids = make_api_request("/features/typeId")
if type_ids:
    print("\n2. TypeIds:")
    if isinstance(type_ids, list):
        print(f"   - {len(type_ids)} typeIds disponibles")
        if len(type_ids) > 0:
            print("   - Ejemplo de typeId:")
            print(f"     {json.dumps(type_ids[0], indent=2)}")
    else:
        print("   - Estructura no es una lista como se esperaba")


Explorando los metadatos de la API para entender su estructura:

1. Qualifiers:
   - 387 qualifiers disponibles
   - Ejemplo de qualifier:
     {
  "": "",
  "QUALIFIER NAME": "Long ball",
  "\ufeffqualifierId": "1"
}

2. TypeIds:
   - 81 typeIds disponibles
   - Ejemplo de typeId:
     {
  "ASSOCIATED qualifierId VALUES": "1, 2, 3, 4, 5, 6, 15, 22, 23, 24, 25, 26, 29, 31, 55, 56, 74, 96, 97, 106, 107, 123, 124, 138, 140, 141, 152, 154, 155, 156, 157, 160, 168, 189, 195, 196, 198, 199, 210, 212, 213, 214, 218, 223, 224, 225, 233, 236, 237, 238, 240, 241, 266, 278, 279, 286, 287, 307, 345, 358, 359, 362, 459",
  "DESCRIPTION": "The attempted delivery of the ball from one player to another player on the same team. A player can use any part of their body (permitted in the laws of the game) to execute a pass. Event categorization includes open play passes, goal kicks, corners and free kicks played as passes. Crosses, keeper throws, and throw ins do not count as passes. outcome: 0 = Unsucc

In [6]:
## 2. Identificación de Endpoints Relevantes

# Mostrar los endpoints más relevantes para nuestro análisis de eventos
relevant_endpoints = [
    {"endpoint": "/competitions", "relevancia": "Alta", "propósito": "Identificar la competición disponible"},
    {"endpoint": "/matches/competition/{competition}", "relevancia": "Alta", "propósito": "Obtener todos los partidos de la competición"},
    {"endpoint": "/match/events/{match_id}", "relevancia": "Alta", "propósito": "Obtener eventos detallados de cada partido"},
    {"endpoint": "/match/base/{match_id}", "relevancia": "Media", "propósito": "Enriquecer los datos con información básica del partido"},
    {"endpoint": "/teams", "relevancia": "Media", "propósito": "Obtener nombres de equipos para enriquecer los datos"},
    {"endpoint": "/match/players/{match_id}", "relevancia": "Baja", "propósito": "Obtener datos de jugadores (no necesario para el objetivo inicial)"},
    {"endpoint": "/match/stats/{match_id}", "relevancia": "Baja", "propósito": "Obtener estadísticas agregadas (no necesario para el objetivo inicial)"}
]

relevant_endpoints_df = pd.DataFrame(relevant_endpoints)
print("Endpoints relevantes para nuestro análisis:")
display(relevant_endpoints_df)

# Realizar una comprobación rápida de los endpoints clave
print("\nComprobación de los endpoints clave:")

# Verificar competiciones
competitions = make_api_request("/competitions")
if competitions:
    print(f"- Competiciones: {len(competitions)} disponibles")

# Verificar equipos
teams = make_api_request("/teams")
if teams:
    print(f"- Equipos: {len(teams)} disponibles")

# Verificar partidos (sin filtrar por competición aún)
matches = make_api_request("/matches")
if matches:
    print(f"- Partidos totales: {len(matches)} disponibles")

Endpoints relevantes para nuestro análisis:


,endpoint,relevancia,propósito
0,/competitions,Alta,Identificar la competición disponible
1,/matches/competition/{competition},Alta,Obtener todos los partidos de la competición
2,/match/events/{match_id},Alta,Obtener eventos detallados de cada partido
3,/match/base/{match_id},Media,Enriquecer los datos con información básica de...
4,/teams,Media,Obtener nombres de equipos para enriquecer los...
5,/match/players/{match_id},Baja,Obtener datos de jugadores (no necesario para ...
6,/match/stats/{match_id},Baja,Obtener estadísticas agregadas (no necesario p...



Comprobación de los endpoints clave:
- Competiciones: 1 disponibles
- Equipos: 36 disponibles
- Partidos totales: 881 disponibles


In [7]:
## 3. Planificación de la Estrategia de Adquisición de Datos

# Definir nuestra estrategia de adquisición
print("\nEstrategia de adquisición de datos:")
print("""
1. Obtener la lista de competiciones (/competitions) - Un solo llamado a la API
2. Obtener todos los partidos de la competición (/matches/competition/{competition}) - Un llamado por competición
3. Para cada partido:
   a. Obtener eventos (/match/events/{match_id}) - Un llamado por partido
   b. Obtener información básica (/match/base/{match_id}) - Un llamado por partido
4. Obtener información de equipos (/teams) - Un solo llamado a la API

Total de llamados a la API = 1 + 1 + (N × 2) + 1 = 3 + 2N
donde N es el número de partidos en la competición
""")

# Estimar el número de llamados a la API
if 'matches' in locals() and matches:
    # Obtener el número aproximado de partidos por competición
    if competitions and len(competitions) > 0:
        comp_id = competitions[0].get('id') or competitions[0].get('competitionId')
        comp_matches = make_api_request(f"/matches/competition/{comp_id}")
        if comp_matches:
            n_matches = len(comp_matches)
            total_calls = 3 + (2 * n_matches)
            print(f"Para {n_matches} partidos en la competición, estimamos aproximadamente {total_calls} llamados a la API")

# Consideraciones sobre límites de tasa
print("\nConsideraciones sobre límites de tasa:")
print("""
La API MPAD Cafecito no especifica explícitamente límites de tasa. Sin embargo, como buena práctica:

1. Implementaremos un control de errores robusto para manejar posibles fallos en las peticiones
2. Podríamos implementar retrasos entre llamadas consecutivas para evitar sobrecargar el servidor
3. Si fuera necesario, podríamos implementar reintentos automáticos con espera exponencial

Al ser una API principalmente para uso educativo, no esperamos límites de tasa estrictos.
""")

# Implementar una función con manejo de errores y reintentos
print("\nFunción mejorada para peticiones a la API con manejo de errores:")

def make_api_request_robust(endpoint, max_retries=3, initial_delay=1):
    """
    Realiza una petición GET a la API con manejo de errores y reintentos.
    
    Args:
        endpoint (str): El endpoint de la API a consultar
        max_retries (int): Número máximo de reintentos
        initial_delay (float): Retraso inicial entre reintentos (segundos)
        
    Returns:
        dict: Los datos recibidos de la API en formato JSON
    """
    url = f"{BASE_URL}{endpoint}"
    
    for attempt in range(max_retries + 1):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            
            # Verificar si la petición fue exitosa
            if response.status_code == 200:
                return response.json()
            
            # Si el error es por límite de tasa, esperar más tiempo
            if response.status_code == 429:
                wait_time = initial_delay * (2 ** attempt)
                print(f"Límite de tasa alcanzado. Esperando {wait_time:.2f} segundos...")
                time.sleep(wait_time)
                continue
                
            # Otros errores
            print(f"Error en la petición (intento {attempt+1}/{max_retries+1}): {response.status_code}")
            print(response.text)
            
            # Si no es el último intento, esperar antes de reintentar
            if attempt < max_retries:
                wait_time = initial_delay * (2 ** attempt)
                print(f"Reintentando en {wait_time:.2f} segundos...")
                time.sleep(wait_time)
            
        except Exception as e:
            print(f"Excepción al realizar la petición (intento {attempt+1}/{max_retries+1}): {str(e)}")
            
            # Si no es el último intento, esperar antes de reintentar
            if attempt < max_retries:
                wait_time = initial_delay * (2 ** attempt)
                print(f"Reintentando en {wait_time:.2f} segundos...")
                time.sleep(wait_time)
    
    # Si llegamos aquí, todos los intentos fallaron
    print(f"Todos los intentos fallaron para el endpoint: {endpoint}")
    return None



Estrategia de adquisición de datos:

1. Obtener la lista de competiciones (/competitions) - Un solo llamado a la API
2. Obtener todos los partidos de la competición (/matches/competition/{competition}) - Un llamado por competición
3. Para cada partido:
   a. Obtener eventos (/match/events/{match_id}) - Un llamado por partido
   b. Obtener información básica (/match/base/{match_id}) - Un llamado por partido
4. Obtener información de equipos (/teams) - Un solo llamado a la API

Total de llamados a la API = 1 + 1 + (N × 2) + 1 = 3 + 2N
donde N es el número de partidos en la competición

Error al realizar la petición: 500
<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>


Consideraciones sobre límites de tasa:

La API MPAD Cafecito no especifica explícitamente límites de tasa.

In [8]:
## 4. Consideraciones Éticas y Legales del Uso de la API

# Descargo de responsabilidad y aviso legal de la API
legal_disclaimer = """
Descargo de responsabilidad y Aviso Legal
La información y los datos presentados a través de esta API han sido obtenidos de fuentes disponibles públicamente en Internet. 
Esta aplicación se ha desarrollado únicamente con fines educativos, en el marco de los programas de Maestría de 
Sports Data Campus (SDC) y no tiene ningún propósito comercial. Se pone a disposición de los usuarios **tal cual**, 
sin ninguna garantía expresa o implícita de ningún tipo, incluyendo pero no limitándose a garantías de idoneidad 
para un propósito particular, exactitud, integridad o disponibilidad. Los derechos de autor, marcas y demás derechos 
de propiedad intelectual pertenecen a sus respectivos titulares. El uso de la información contenida en esta API se 
realiza bajo el principio del **uso justo** y para fines de enseñanza, investigación y análisis académico.

**IMPORTANTE:**
* El usuario es responsable de cumplir con todas las leyes y regulaciones aplicables en su jurisdicción en relación con el uso de los datos.
* Se recomienda que, en caso de querer utilizar estos datos para otros fines (por ejemplo, publicaciones, proyectos comerciales 
  o difusión pública), se consulte primero con los titulares de los derechos de la información original y se obtengan las 
  autorizaciones correspondientes.
* La administración de este proyecto no asume responsabilidad alguna por daños o perjuicios derivados del uso de la 
  información aquí dispuesta.
"""

print("Descargo de responsabilidad y aviso legal de la API:")
print(legal_disclaimer)

# Nuestras consideraciones éticas y legales
ethical_considerations = """
Consideraciones éticas y legales para nuestro uso de la API:

1. Uso educativo: Estamos utilizando estos datos únicamente con fines educativos y de investigación académica.

2. Privacidad de datos: Aunque los datos parecen ser públicos, debemos tener cuidado con cualquier información personal 
   que pueda estar presente. No publicaremos ni compartiremos ningún dato que pueda comprometer la privacidad.

3. Atribución: Reconocemos que los datos originales pertenecen a sus respectivos propietarios. En cualquier publicación 
   o presentación que utilice estos datos, debemos atribuir adecuadamente la fuente.

4. Uso justo: Nuestro uso se limita al análisis educativo y no comercial, lo que se alinea con el principio de "uso justo".

5. Respeto a los límites de la API: Implementamos una estrategia de adquisición de datos que respeta los recursos del servidor,
   evitando sobrecargarlo con demasiadas peticiones simultáneas.

6. No redistribución: No redistribuiremos los datos crudos obtenidos de la API sin permiso explícito.

7. Almacenamiento seguro: Almacenaremos los datos de manera segura y solo durante el tiempo necesario para nuestro análisis.

8. Interpretación responsable: Nos comprometemos a hacer interpretaciones responsables de los datos, reconociendo sus 
   limitaciones y evitando conclusiones injustificadas.
"""

print("\nNuestras consideraciones éticas y legales:")
print(ethical_considerations)

Descargo de responsabilidad y aviso legal de la API:

Descargo de responsabilidad y Aviso Legal
La información y los datos presentados a través de esta API han sido obtenidos de fuentes disponibles públicamente en Internet. 
Esta aplicación se ha desarrollado únicamente con fines educativos, en el marco de los programas de Maestría de 
Sports Data Campus (SDC) y no tiene ningún propósito comercial. Se pone a disposición de los usuarios **tal cual**, 
sin ninguna garantía expresa o implícita de ningún tipo, incluyendo pero no limitándose a garantías de idoneidad 
para un propósito particular, exactitud, integridad o disponibilidad. Los derechos de autor, marcas y demás derechos 
de propiedad intelectual pertenecen a sus respectivos titulares. El uso de la información contenida en esta API se 
realiza bajo el principio del **uso justo** y para fines de enseñanza, investigación y análisis académico.

**IMPORTANTE:**
* El usuario es responsable de cumplir con todas las leyes y regulaciones

### b) Implementación de la Adquisición de Datos (30 puntos)

* Desarrolle scripts para interactuar con la API o cargar los datos utilizando las bibliotecas apropiadas (requests, pandas, etc.).
* Implemente manejo de autenticación si es necesario.
* Incluya manejo de errores y reintentos.
* Asegúrese de respetar los límites de tasa y políticas de uso de la API.

# Competiciones

In [9]:
# base_url = "https://api-cafecito.onrender.com"
# base_url = "http://127.0.0.1:5000"
headers = {"Authorization": f"Bearer {AUTH_TOKEN}"}

# Obtener la lista completa de competiciones (sin parámetros)
url = f"{BASE_URL}/competitions"
response = requests.get(url, headers=headers)
print("Lista completa de competiciones:")
print(response.json())

Lista completa de competiciones:
[{'competition': 'Europe-Champions-League-2024-2025', 'regionId': 250, 'seasonId': 10456, 'seasonName': '2024/2025', 'stageId': 23663, 'stageName': 'Champions League', 'tournamentId': 12, 'tournamentName': 'Champions League'}]


# Partidos Champions League 24/25

In [10]:
# 1. Obtener partidos de una competición específica
competition = "Europe-Champions-League-2024-2025"
response = requests.get(f"{BASE_URL}/matches/competition/{competition}", headers=headers)

# 2. Convertir la respuesta JSON a un DataFrame
df_partidos = pd.DataFrame(response.json())

# 3. Verificar si la columna match_id existe en el DataFrame
if "match_id" not in df_partidos.columns:
    raise ValueError("La columna 'match_id' no está presente en los datos.")

# 4. Filtrar partidos sin match_id
df_partidos = df_partidos.dropna(subset=["match_id"])

# 5. Convertir match_id a enteros
df_partidos["match_id"] = df_partidos["match_id"].astype(int)

# 6. Mostrar las primeras filas del DataFrame filtrado
df_partidos.head(100)

,region,tournament,season,stage,competition,date,time_or_status,home_team,away_team,match_id,home_score,away_score,home_odds,draw_odds,away_odds
0,250,12,10456,23663,Europe-Champions-League-2024-2025,"Tuesday, Sep 17 2024",FT,Juventus,PSV Eindhoven,1866178,3,1,NaN,NaN,NaN
2,250,12,10456,24083,europe-champions-league-2024-2025,"Tuesday, Apr 08 2025",20:00,Bayern Munich,Inter,1894448,-,-,1.67,4.0,5.0
3,250,12,10456,24083,europe-champions-league-2024-2025,"Wednesday, Mar 05 2025",FT,Benfica,Barcelona,1891170,0,1,NaN,NaN,NaN
4,250,12,10456,24083,europe-champions-league-2024-2025,"Wednesday, Mar 05 2025",FT,Bayern Munich,Bayer Leverkusen,1891166,3,0,NaN,NaN,NaN
5,250,12,10456,24083,europe-champions-league-2024-2025,"Wednesday, Mar 05 2025",FT,Paris Saint-Germain,Liverpool,1891169,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,250,12,10456,23663,europe-champions-league-2024-2025,"Thursday, Sep 19 2024",FT,Feyenoord,Bayer Leverkusen,1866157,0,4,NaN,NaN,NaN
97,250,12,10456,23663,europe-champions-league-2024-2025,"Thursday, Sep 19 2024",FT,FK Crvena Zvezda,Benfica,1866185,1,2,NaN,NaN,NaN
98,250,12,10456,23663,europe-champions-league-2024-2025,"Wednesday, Oct 02 2024",FT,Aston Villa,Bayern Munich,1866119,1,0,NaN,NaN,NaN
99,250,12,10456,23663,europe-champions-league-2024-2025,"Wednesday, Oct 02 2024",FT,RB Leipzig,Juventus,1866121,2,3,NaN,NaN,NaN


# Jugadores

In [11]:
# 4. Extraer todos los match_id únicos y válidos
valid_match_ids = df_partidos["match_id"].dropna().unique()

# 5. Mostrar la cantidad de partidos válidos
print(f"Se encontraron {len(valid_match_ids)} partidos válidos con match_id.")

# 6. Lista para almacenar los jugadores de cada partido
players_data = []

# 7. Iterar sobre los match_id válidos
for match_id in valid_match_ids:
    url = f"{BASE_URL}/match/players/{match_id}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        players = response.json()
        
        # Procesar jugadores del equipo local
        for player in players.get("homePlayers", []):
            players_data.append({
                "match_id": match_id,
                "team": "home",
                "player_id": player.get("playerId"),
                "player_name": player.get("playerName"),
                "jersey_number": player.get("jerseyNumber"),
                "formation_slot": player.get("formationSlot"),
                "match_start": player.get("matchStart"),
                "team_id": player.get("teamId"),
                # Agrega más campos si es necesario
            })
        
        # Procesar jugadores del equipo visitante
        for player in players.get("awayPlayers", []):
            players_data.append({
                "match_id": match_id,
                "team": "away",
                "player_id": player.get("playerId"),
                "player_name": player.get("playerName"),
                "jersey_number": player.get("jerseyNumber"),
                "formation_slot": player.get("formationSlot"),
                "match_start": player.get("matchStart"),
                "team_id": player.get("teamId"),
                # Agrega más campos si es necesario
            })
    else:
        print(f"Error {response.status_code} para el partido {match_id}: {response.text}")

# 8. Convertir la lista de jugadores a un DataFrame
df_jugadores = pd.DataFrame(players_data)

# 9. Mostrar las primeras filas del DataFrame
df_jugadores.head(100)

Se encontraron 184 partidos válidos con match_id.
Error 500 para el partido 1894448: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Error 500 para el partido 1891170: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Error 500 para el partido 1891166: <!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

Error 500 para el partido 1891169: <!doctype ht

,match_id,team,player_id,player_name,jersey_number,formation_slot,match_start,team_id
0,1866178,home,319696,Michele Di Gregorio,29,1.0,1,87
1,1866178,home,391836,Pierre Kalulu,15,2.0,1,87
2,1866178,home,423440,Andrea Cambiaso,27,3.0,1,87
3,1866178,home,273539,Manuel Locatelli,5,4.0,1,87
4,1866178,home,439909,Federico Gatti,4,5.0,1,87
...,...,...,...,...,...,...,...,...
95,1888117,home,338275,Weston McKennie,16,10.0,1,87
96,1888117,home,494525,Kenan Yildiz,10,11.0,1,87
97,1888117,home,78880,Mattia Perin,1,0.0,0,87
98,1888117,home,83981,Carlo Pinsoglio,23,0.0,0,87


# Equipos


In [12]:
# 3. Obtener teams.csv
url_teams = f"{BASE_URL}/teams"
response = requests.get(url_teams, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir directamente la respuesta JSON a un DataFrame
    df_equipos = pd.DataFrame(response.json())

else:
    print("Error:", response.status_code, response.json())

df_equipos.head()

,countryCode,countryName,imageUrl,matchId,teamId,teamName,﻿
0,at,Austria,https://d2zywfiolv4f83.cloudfront.net/img/team...,1866220,361,Salzburg,0
1,de,Germany,https://d2zywfiolv4f83.cloudfront.net/img/team...,1866237,41,VfB Stuttgart,1
2,nl,Netherlands,https://d2zywfiolv4f83.cloudfront.net/img/team...,1866209,129,PSV Eindhoven,2
3,de,Germany,https://d2zywfiolv4f83.cloudfront.net/img/team...,1866154,36,Bayer Leverkusen,3
4,gb-eng,England,https://d2zywfiolv4f83.cloudfront.net/img/team...,1866166,13,Arsenal,4


# Eventos


In [13]:
# Lista para almacenar todos los eventos
events_data = []

# Iterar sobre los match_ids válidos
for match_id in valid_match_ids:
    try:
        # URL para obtener eventos del partido
        url = f"{BASE_URL}/match/events/{match_id}"
        
        # Realizar la solicitud
        response = requests.get(url, headers=headers)
        
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            result = response.json()
            
            # Procesar cada evento del partido
            for event in result.get("events", []):
                # Crear un diccionario con la información del evento
                event_info = event.copy()
                event_info['match_id'] = match_id
                events_data.append(event_info)
            
            print(f"Eventos obtenidos para el partido {match_id}")
        else:
            print(f"Error al obtener eventos para el partido {match_id}: {response.status_code}")
        
        # Añadir un pequeño retraso para evitar sobrecargar la API
        time.sleep(0.5)
    
    except Exception as e:
        print(f"Excepción al procesar el partido {match_id}: {e}")

# Crear DataFrame de eventos
df_events = pd.DataFrame(events_data)

df_events.head()

Eventos obtenidos para el partido 1866178
Error al obtener eventos para el partido 1894448: 500
Error al obtener eventos para el partido 1891170: 500
Error al obtener eventos para el partido 1891166: 500
Error al obtener eventos para el partido 1891169: 500
Error al obtener eventos para el partido 1891178: 500
Error al obtener eventos para el partido 1888120: 500
Error al obtener eventos para el partido 1891167: 500
Error al obtener eventos para el partido 1891165: 500
Eventos obtenidos para el partido 1888123
Eventos obtenidos para el partido 1888117
Eventos obtenidos para el partido 1888119
Eventos obtenidos para el partido 1888115
Eventos obtenidos para el partido 1888111
Eventos obtenidos para el partido 1888113
Eventos obtenidos para el partido 1888116
Error al obtener eventos para el partido 1888112: 500
Error al obtener eventos para el partido 1888114: 500
Error al obtener eventos para el partido 1888126: 500
Error al obtener eventos para el partido 1888118: 500
Error al obtener

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,qualifiers,satisfiedEventsTypes,second,...,blockedX,blockedY,goalMouthY,goalMouthZ,isShot,relatedEventId,relatedPlayerId,isGoal,cardType,isOwnGoal
0,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndX', 'value':...","[91, 117, 30, 35, 38, 215, 218]",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndY', 'value':...","[91, 117, 30, 35, 37, 215, 218]",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Zone', 'value': 56}...","[91, 117, 30, 35, 38, 215, 218]",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_events.head(100)

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,qualifiers,satisfiedEventsTypes,second,...,blockedX,blockedY,goalMouthY,goalMouthZ,isShot,relatedEventId,relatedPlayerId,isGoal,cardType,isOwnGoal
0,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndX', 'value':...","[91, 117, 30, 35, 38, 215, 218]",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndY', 'value':...","[91, 117, 30, 35, 37, 215, 218]",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Zone', 'value': 56}...","[91, 117, 30, 35, 38, 215, 218]",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,60,4,2.721836e+09,False,4,"{'displayName': 'Unsuccessful', 'value': 0}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Defensive', 'value'...","[198, 200]",44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,49,4,2.721836e+09,True,4,"{'displayName': 'Unsuccessful', 'value': 0}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Angle', 'value': 21...","[91, 120, 139, 128, 36, 37, 217, 218]",44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,62,4,2.721836e+09,True,4,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Zone', 'value': 56}...","[91, 117, 30, 36, 38, 215, 218]",58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,63,5,2.721836e+09,True,5,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndY', 'value':...","[91, 117, 30, 36, 38, 215, 218]",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Función para procesar los qualifiers
def parse_qualifiers(event):
    # Extraer el eventId del evento
    event_id = event.get('id')
    
    # Lista para almacenar los qualifiers parseados
    parsed_qualifiers = []
    
    # Procesar cada qualifier
    for qualifier in event.get('qualifiers', []):
        # Crear un diccionario con la información del qualifier
        parsed_qualifier = {
            'id': event_id,
            'qualifier_name': qualifier['type']['displayName'],
            'qualifier_type_value': qualifier['type']['value'],
            'qualifier_value': qualifier.get('value')
        }
        parsed_qualifiers.append(parsed_qualifier)
    
    return parsed_qualifiers

# Procesar qualifiers de todos los eventos
qualifiers_data = []
for _, evento in df_events.iterrows():
    # Parsear los qualifiers del evento
    qualifiers_parseados = parse_qualifiers(evento)
    qualifiers_data.extend(qualifiers_parseados)

# Crear DataFrame de qualifiers
df_qualifiers = pd.DataFrame(qualifiers_data)

df_qualifiers.head(100)

,id,qualifier_name,qualifier_type_value,qualifier_value
0,2.721835e+09,PassEndX,140,32.5
1,2.721835e+09,Angle,213,3.33
2,2.721835e+09,Length,212,18.6
3,2.721835e+09,StandingSave,178,None
4,2.721835e+09,PassEndY,141,44.9
...,...,...,...,...
95,2.721835e+09,Zone,56,Back
96,2.721835e+09,Length,212,5.6
97,2.721835e+09,Length,212,18.6
98,2.721835e+09,Zone,56,Back


### c) Limpieza y Preprocesamiento de Datos (10 puntos)

* Limpie los datos adquiridos (manejo de valores faltantes, corrección de formatos, etc.).
* Realice transformaciones necesarias para el análisis (creación de nuevas columnas, agregaciones, etc.).
* Guarde los datos procesados en un formato estructurado (CSV, JSON, etc.).

In [16]:
# Método para agregar qualifiers a df_events
def add_qualifiers_to_events(df_events, df_qualifiers):
    # Crear una copia del DataFrame original
    df_events = df_events.copy()
    
    # Crear columna de características
    df_events['características'] = ''
    
    # Diccionarios para manejar diferentes tipos de qualifiers
    caracteristicas = {}
    numeros = {}
    
    # Iterar sobre los eventos y procesar los qualifiers correspondientes
    for index, evento in df_events.iterrows():
        # Filtrar qualifiers para este evento específico
        qualifiers_evento = df_qualifiers[df_qualifiers['id'] == evento['id']]
        
        # Separar qualifiers en características y números
        for _, qualifier in qualifiers_evento.iterrows():
            nombre = qualifier.qualifier_name
            valor = qualifier.qualifier_value
            
            # Clasificar qualifiers
            if valor in ['0', None]:
                # Características textuales o booleanas
                if nombre not in caracteristicas:
                    caracteristicas[nombre] = set()
                caracteristicas[nombre].add(str(valor))
            else:
                # Valores numéricos
                numeros[nombre] = valor
        
        # Añadir características
        caract_texto = ', '.join([f"{k}" for k, v in caracteristicas.items() if v != {'0'}])
        df_events.at[index, 'características'] = caract_texto
        
        # Añadir columnas numéricas
        for nombre, valor in numeros.items():
            df_events.at[index, nombre] = valor
        
        # Limpiar diccionarios para el siguiente evento
        caracteristicas.clear()
        numeros.clear()
    
    return df_events

# Aplicar la función
df_events = add_qualifiers_to_events(df_events, df_qualifiers)

df_events.head(100)

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,qualifiers,satisfiedEventsTypes,second,...,PlayerCaughtOffside,LeadingToGoal,JerseyNumber,PlayerPosition,FormationSlot,TeamPlayerFormation,CaptainPlayerId,TeamFormation,InvolvedPlayers,Foul
0,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,2.721835e+09,False,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}",[],[],0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndX', 'value':...","[91, 117, 30, 35, 38, 215, 218]",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndY', 'value':...","[91, 117, 30, 35, 37, 215, 218]",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,2.721835e+09,True,0,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Zone', 'value': 56}...","[91, 117, 30, 35, 38, 215, 218]",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,60,4,2.721836e+09,False,4,"{'displayName': 'Unsuccessful', 'value': 0}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Defensive', 'value'...","[198, 200]",44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,49,4,2.721836e+09,True,4,"{'displayName': 'Unsuccessful', 'value': 0}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Angle', 'value': 21...","[91, 120, 139, 128, 36, 37, 217, 218]",44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,62,4,2.721836e+09,True,4,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'Zone', 'value': 56}...","[91, 117, 30, 36, 38, 215, 218]",58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,63,5,2.721836e+09,True,5,"{'displayName': 'Successful', 'value': 1}","{'displayName': 'FirstHalf', 'value': 1}","[{'type': {'displayName': 'PassEndY', 'value':...","[91, 117, 30, 36, 38, 215, 218]",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Función para limpiar el DataFrame de eventos
def limpiar_df_events(df_events):
    # Crear una copia del DataFrame
    df_limpio = df_events.copy()
    
    # Columnas a eliminar
    columnas_eliminar = [
        col for col in df_limpio.columns 
        if any(substring in col for substring in [
            'relatedEventId', 
            'relatedPlayerId', 
            'qualifiers', 
            'GoalMouthY', 
            'GoalMouthZ', 
            'satisfiedEventsTypes', 
            'PassEndX', 
            'PassEndY'
        ])
    ]
    
    # Eliminar columnas
    df_limpio = df_limpio.drop(columns=columnas_eliminar)
    
    # Función para extraer displayName de diccionarios
    def extraer_display_name(valor):
        if isinstance(valor, dict) and 'displayName' in valor:
            return valor['displayName']
        return valor
    
    # Columnas a procesar
    columnas_a_procesar = ['outcomeType', 'period', 'type']
    
    # Aplicar extracción de displayName
    for col in columnas_a_procesar:
        if col in df_limpio.columns:
            df_limpio[col] = df_limpio[col].apply(extraer_display_name)
    
    return df_limpio

# Aplicar la limpieza
df_events = limpiar_df_events(df_events)

df_events.head()

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,second,teamId,type,...,PlayerCaughtOffside,LeadingToGoal,JerseyNumber,PlayerPosition,FormationSlot,TeamPlayerFormation,CaptainPlayerId,TeamFormation,InvolvedPlayers,Foul
0,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,129,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,2.721835e+09,True,0,Successful,FirstHalf,0.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,2.721835e+09,True,0,Successful,FirstHalf,2.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,2.721835e+09,True,0,Successful,FirstHalf,5.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Asegurar que los IDs sean numéricos
# Conversión de IDs en df_events
df_events['playerId'] = pd.to_numeric(df_events['playerId'], errors='coerce')
df_events['teamId'] = pd.to_numeric(df_events['teamId'], errors='coerce')

# Conversión de IDs en df_jugadores
df_jugadores['player_id'] = pd.to_numeric(df_jugadores['player_id'], errors='coerce')

# Conversión de IDs en df_equipos
df_equipos['teamId'] = pd.to_numeric(df_equipos['teamId'], errors='coerce')

# Merge para añadir columna de jugador
df_events = df_events.merge(
    df_jugadores[['player_id', 'player_name', 'formation_slot', 'jersey_number']], 
    left_on='playerId', 
    right_on='player_id', 
    how='left'
)

# Merge para añadir columna de equipo
df_events = df_events.merge(
    df_equipos[['teamId', 'teamName']], 
    left_on='teamId', 
    right_on='teamId', 
    how='left'
)
df_events.head(100)

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,second,teamId,type,...,TeamPlayerFormation,CaptainPlayerId,TeamFormation,InvolvedPlayers,Foul,player_id,player_name,formation_slot,jersey_number,teamName
0,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juventus
1,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,129,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PSV Eindhoven
2,3,0,2.721835e+09,True,0,Successful,FirstHalf,0.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,315369.0,Dusan Vlahovic,9.0,9.0,Juventus
3,3,0,2.721835e+09,True,0,Successful,FirstHalf,0.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,315369.0,Dusan Vlahovic,0.0,9.0,Juventus
4,3,0,2.721835e+09,True,0,Successful,FirstHalf,0.0,87,Pass,...,NaN,NaN,NaN,NaN,NaN,315369.0,Dusan Vlahovic,9.0,9.0,Juventus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,14,0,2.721835e+09,True,0,Unsuccessful,FirstHalf,41.0,129,BallTouch,...,NaN,NaN,NaN,NaN,NaN,444164.0,Matteo Dams,3.0,32.0,PSV Eindhoven
96,14,0,2.721835e+09,True,0,Unsuccessful,FirstHalf,41.0,129,BallTouch,...,NaN,NaN,NaN,NaN,NaN,444164.0,Matteo Dams,3.0,32.0,PSV Eindhoven
97,14,0,2.721835e+09,True,0,Unsuccessful,FirstHalf,41.0,129,BallTouch,...,NaN,NaN,NaN,NaN,NaN,444164.0,Matteo Dams,3.0,32.0,PSV Eindhoven
98,14,0,2.721835e+09,True,0,Unsuccessful,FirstHalf,41.0,129,BallTouch,...,NaN,NaN,NaN,NaN,NaN,444164.0,Matteo Dams,3.0,32.0,PSV Eindhoven


In [30]:
# Convertir match_id a numérico para asegurar consistencia
df_events['match_id'] = pd.to_numeric(df_events['match_id'], errors='coerce')
df_partidos['match_id'] = pd.to_numeric(df_partidos['match_id'], errors='coerce')

# Crear la columna de partido con el formato especificado
df_events = df_events.merge(
    df_partidos[['match_id', 'home_team', 'away_team', 'home_score', 'away_score']], 
    left_on='match_id', 
    right_on='match_id', 
    how='left'
)

# Formatear la columna 'match'
df_events['match'] = (
    df_events['home_team'] + ' ' + 
    df_events['home_score'].astype(str) + 
    ' - ' + 
    df_events['away_score'].astype(str) + ' ' + 
    df_events['away_team']
)

# Eliminar columnas intermedias
df_events = df_events.drop(columns=['home_team', 'away_team', 'home_score', 'away_score'])
df_events.to_csv('eventos.csv', index=False)


df_events.head(100)

,eventId,expandedMinute,id,isTouch,minute,outcomeType,period,second,teamId,type,...,CaptainPlayerId,TeamFormation,InvolvedPlayers,Foul,player_id,player_name,formation_slot,jersey_number,teamName,match
0,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juventus,Juventus 3 - 1 PSV Eindhoven
1,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juventus,Juventus 3 - 1 PSV Eindhoven
2,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juventus,Juventus 3 - 1 PSV Eindhoven
3,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,87,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juventus,Juventus 3 - 1 PSV Eindhoven
4,2,0,2.721835e+09,False,0,Successful,FirstHalf,0.0,129,Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PSV Eindhoven,Juventus 3 - 1 PSV Eindhoven
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,0,2.721835e+09,True,0,Successful,FirstHalf,5.0,87,Pass,...,NaN,NaN,NaN,NaN,423440.0,Andrea Cambiaso,0.0,27.0,Juventus,Juventus 3 - 1 PSV Eindhoven
96,5,0,2.721835e+09,True,0,Successful,FirstHalf,5.0,87,Pass,...,NaN,NaN,NaN,NaN,423440.0,Andrea Cambiaso,3.0,27.0,Juventus,Juventus 3 - 1 PSV Eindhoven
97,5,0,2.721835e+09,True,0,Successful,FirstHalf,5.0,87,Pass,...,NaN,NaN,NaN,NaN,423440.0,Andrea Cambiaso,3.0,27.0,Juventus,Juventus 3 - 1 PSV Eindhoven
98,5,0,2.721835e+09,True,0,Successful,FirstHalf,5.0,87,Pass,...,NaN,NaN,NaN,NaN,423440.0,Andrea Cambiaso,3.0,27.0,Juventus,Juventus 3 - 1 PSV Eindhoven
